In [1]:
import torch
import torch.nn as nn
import random
import gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros import SuperMarioBrosEnv
from tqdm import tqdm
import pickle 
import gym
import numpy as np
import collections 
import cv2
import matplotlib.pyplot as plt
import time
import datetime
import json
from toolkit.gym_env import *
from toolkit.action_utils import *
# from toolkit.marlios_lstm_action import *
from toolkit.train_marlios import *
from toolkit.constants import *
from toolkit.train_test_samples import *
import warnings

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


In [3]:
training_mode=True
pretrained=False
lr=0.0001
gamma=0.90
exploration_decay=0.995
exploration_min=0.02
ep_per_stat = 100
exploration_max = 1
lr_decay = 0.99
mario_env='SuperMarioBros-1-1-v0'
action_space=SIMPLE_MOVEMENT
num_episodes=1000
run_id='1683228531'
n_actions=len(SIMPLE_MOVEMENT)
debug = True
name=None
max_time_per_ep = 500
device='cpu'
log=True
add_sufficient = False
training_stage = "train"
validate_every = 50
run_id = run_id or generate_epoch_time_id()
env = gym.make(mario_env)
env = make_env(env, ACTION_SPACE)
agent = DQNAgent(
                    state_space=env.observation_space.shape,
                    action_space=action_space,
                    max_memory_size=30000,
                    batch_size=n_actions,
                    gamma=gamma,
                    lr=lr,
                    dropout=None,
                    exploration_max=exploration_max,
                    exploration_min=exploration_min,
                    exploration_decay=exploration_decay,
                    double_dq=True,
                    pretrained=pretrained,
                    run_id=run_id,
                    n_actions=n_actions,
                    device=device,
                    init_max_time=max_time_per_ep,
                    training_stage=training_stage,
                    add_sufficient=add_sufficient
                    )

In [12]:
env.reset()
for i in range(4): 
    state_next, _, _, _ = env.step(7)
    
state = torch.Tensor([state_next])

In [15]:
agent.subsample_actions()
cur_action_space = agent.cur_action_space

big_conv_out = agent.local_net.conv(state).view(state.size()[0], -1)
conv_out = agent.local_net.conv_to_32(big_conv_out)

batched_conv_out = conv_out.reshape(conv_out.shape[0], 1, conv_out.shape[-1]).repeat(1, cur_action_space.shape[-2], 1)

latent_actions = agent.local_net.action_fc(cur_action_space)

batched_actions = torch.cat((batched_conv_out, latent_actions), dim=2)

out =  torch.flatten(agent.local_net.fc(batched_actions), start_dim=1)
print(out)

tensor([[1.1671, 1.5383, 0.8406, 0.1926, 0.9249, 0.5038, 0.7398]],
       grad_fn=<ReshapeAliasBackward0>)


In [14]:
print(out)

tensor([[1.1671, 0.8406, 0.9249, 0.5038, 0.7398, 0.1926, 1.5383]],
       grad_fn=<ReshapeAliasBackward0>)
